In [1]:
import pandas as pd
import polars as pl
from utils.util import generate_postgres_ddl, df_to_db, read_database, engine
from config import sheet_config_travel_journey
from openpyxl.utils import column_index_from_string
import openpyxl
import time
import datetime
import os
from pyxlsb import open_workbook
import sqlite3

In [3]:
query = """
    with
        grouping_data as (
            select
                kode_titik,
                case
                    when asal_tempat in ('-', '0') then 'Tempat Tidak Diketahui'
                    when asal_tempat in ('tempatkerja') then 'Tempat Kerja'
                    when asal_tempat in ('tempatpendidikan') then 'Tempat Pendidikan'
                    when asal_tempat in ('tempatperbelanjaan') then 'Tempat Perbelanjaan'
                    when asal_tempat in ('tempatrekreasi') then 'Tempat Rekreasi'
                    when asal_tempat in ('tempattinggal') then 'Tempat Tinggal'
                    when asal_tempat in ('tempatusaha') then 'Tempat Usaha'
                end as asal_tempat,
                case
                    when tujuan_tempat in ('-', '0') then 'Tempat Tidak Diketahui'
                    when tujuan_tempat in ('tempatkerja') then 'Tempat Kerja'
                    when tujuan_tempat in ('tempatpendidikan') then 'Tempat Pendidikan'
                    when tujuan_tempat in ('tempatperbelanjaan') then 'Tempat Perbelanjaan'
                    when tujuan_tempat in ('tempatrekreasi') then 'Tempat Rekreasi'
                    when tujuan_tempat in ('tempattinggal') then 'Tempat Tinggal'
                    when tujuan_tempat in ('tempatusaha') then 'Tempat Usaha'
                end as tujuan_tempat,
                upper(trim(asal_kab_kota)) as asal_kab_kota_original,
                upper(trim(tujuan_kab_kota)) as tujuan_kab_kota_original,
                case
                    when upper(trim(asal_kab_kota)) in ('BALI', 'BANDAR LAMPUNG', 'BENGKULU', 'JAMBI', 'LAMPUNG', 'RIAU',
                                        'SUMATERA BARAT', 'SUMATERA SELATAN', 'SUMATERA UTARA', 'KALIMANTAN TENGAH',
                                        'SULAWESI SELATAN', 'BALIKPAPAN', 'NUSA TENGGARA BARAT', 'KALIMANTAN SELATAN',
                                        'KALIMANTAN TENGAH', 'KAB. BULELENG') then 'EKSTERNAL'
                    else REPLACE(asal_kab_kota, 'KAB. ', '')
                end as asal_kab_kota,
                case
                when upper(trim(tujuan_kab_kota)) in ('BALI', 'BANDAR LAMPUNG', 'BENGKULU', 'JAMBI', 'LAMPUNG', 'RIAU',
                                    'SUMATERA BARAT', 'SUMATERA SELATAN', 'SUMATERA UTARA', 'KALIMANTAN TENGAH',
                                    'SULAWESI SELATAN', 'BALIKPAPAN', 'NUSA TENGGARA BARAT', 'KALIMANTAN SELATAN',
                                    'KALIMANTAN TENGAH', 'ACEH', 'KOTA MEDAN', 'IKN', 'KOTA PALEMBANG',
                                    'BANGKA BELITUNG', 'KOTA PEKANBARU', 'NUSA TENGGARA TIMUR', 'KAB. LAMPUNG SELATAN',
                                    'KAB. LAMPUNG BARAT', 'KOTA DENPASAR', 'KOTA MATARAM', 'KAB. SUMBAWA', 'KAB. SIAK',
                                    'KOTA PAGAR ALAM') then 'EKSTERNAL'
                else REPLACE(tujuan_kab_kota, 'KAB. ', '')
                end as tujuan_kab_kota,
                r_asal.latitude as asal_latitude,
                r_asal.longitude as asal_longitude,
                r_tujuan.latitude as tujuan_latitude,
                r_tujuan.longitude as tujuan_longitude
            from survey_asal_tujuan s
            left join regencies r_asal
            on REPLACE(upper(REPLACE(upper(s.asal_kab_kota), 'ADM. ', '')), 'KAB. ', 'KABUPATEN ') = r_asal.name
            left join regencies r_tujuan
            on REPLACE(upper(REPLACE(upper(s.tujuan_kab_kota), 'ADM. ', '')), 'KAB. ', 'KABUPATEN ') = r_tujuan.name
        ),
        agg_data as (
            select
                upper(kode_titik) as kode_titik,
                COUNT(1) OVER (PARTITION BY kode_titik) AS count_data,
                COUNT(1) OVER (PARTITION BY kode_titik, asal_tempat) AS count_origin,
                COUNT(1) OVER (PARTITION BY kode_titik, tujuan_tempat) AS count_dest,
                COUNT(1) OVER (PARTITION BY kode_titik, asal_tempat, tujuan_tempat) AS count_origin_dest,
                COUNT(1) OVER (PARTITION BY kode_titik, asal_kab_kota, tujuan_kab_kota) AS count_origin_dest_kab_kota,
                asal_tempat,
                tujuan_tempat,
                asal_kab_kota,
                asal_kab_kota_original,
                tujuan_kab_kota,
                tujuan_kab_kota_original,
                asal_latitude,
                asal_longitude,
                tujuan_latitude,
                tujuan_longitude
        from grouping_data
    )

    select
        distinct
        upper(kode_titik) as kode_titik,
        asal_tempat,
        ROUND(((count_origin::float / count_data::float) * 100)::numeric, 2) as precentage_asal_tempat,
        tujuan_tempat,
        ROUND(((count_dest::float / count_data::float) * 100)::numeric, 2) as precentage_tujuan_tempat,
        asal_kab_kota,
        tujuan_kab_kota,
        count_origin_dest_kab_kota as _count_origin_dest_kab_kota,
        case
            when count_origin_dest_kab_kota <= 10 then 10
            when count_origin_dest_kab_kota > 10 and count_origin_dest_kab_kota <= 20 then 20
            when count_origin_dest_kab_kota > 20 and count_origin_dest_kab_kota <= 30 then 30
            when count_origin_dest_kab_kota > 30 and count_origin_dest_kab_kota <= 40 then 40
            when count_origin_dest_kab_kota > 40 and count_origin_dest_kab_kota <= 50 then 50
            when count_origin_dest_kab_kota > 50 and count_origin_dest_kab_kota <= 60 then 60
            when count_origin_dest_kab_kota > 60 and count_origin_dest_kab_kota <= 70 then 70
            when count_origin_dest_kab_kota > 70 and count_origin_dest_kab_kota <= 80 then 80
            when count_origin_dest_kab_kota > 80 and count_origin_dest_kab_kota <= 90 then 90
            when count_origin_dest_kab_kota > 90 and count_origin_dest_kab_kota <= 100 then 100
            when count_origin_dest_kab_kota > 100 and count_origin_dest_kab_kota <= 500 then 500
            when count_origin_dest_kab_kota > 500 and count_origin_dest_kab_kota <= 1000 then 1000
            when count_origin_dest_kab_kota > 1000 and count_origin_dest_kab_kota <= 1500 then 1500
            when count_origin_dest_kab_kota > 1500 and count_origin_dest_kab_kota <= 2000 then 2000
        end count_origin_dest_kab_kota,
        case
            when asal_kab_kota_original in ('BANDAR LAMPUNG', 'BENGKULU', 'JAMBI', 'LAMPUNG', 'RIAU', 'SUMATERA BARAT',
                     'SUMATERA SELATAN', 'SUMATERA UTARA', 'ACEH', 'BANGKA BELITUNG', 'KAB. LAMPUNG BARAT',
                     'KAB. LAMPUNG SELATAN', 'KAB. SIAK', 'KOTA MEDAN', 'KOTA PAGAR ALAM', 'KOTA PALEMBANG',
                     'KOTA PEKANBARU') then -5.933002183915104
            WHEN asal_kab_kota_original in ('BALI', 'KAB. BULELENG', 'NUSA TENGGARA BARAT', 'KAB. SUMBAWA', 'KOTA DENPASAR',
                     'KOTA MATARAM', 'NUSA TENGGARA BARAT', 'NUSA TENGGARA TIMUR') then -8.142900298290554
            WHEN asal_kab_kota_original in ('IKN', 'KALIMANTAN SELATAN', 'BALIKPAPAN', 'KALIMANTAN TENGAH',
                     'SULAWESI SELATAN') then -7.219475761110419
            else asal_latitude
        end as asal_latitude,
        case
            when asal_kab_kota_original in ('BANDAR LAMPUNG', 'BENGKULU', 'JAMBI', 'LAMPUNG', 'RIAU', 'SUMATERA BARAT',
                     'SUMATERA SELATAN', 'SUMATERA UTARA', 'ACEH', 'BANGKA BELITUNG', 'KAB. LAMPUNG BARAT',
                     'KAB. LAMPUNG SELATAN', 'KAB. SIAK', 'KOTA MEDAN', 'KOTA PAGAR ALAM', 'KOTA PALEMBANG',
                     'KOTA PEKANBARU') then 105.99945230749717
            WHEN asal_kab_kota_original in ('BALI', 'KAB. BULELENG', 'NUSA TENGGARA BARAT', 'KAB. SUMBAWA', 'KOTA DENPASAR',
                     'KOTA MATARAM', 'NUSA TENGGARA BARAT', 'NUSA TENGGARA TIMUR') then 114.40067867683435
            WHEN asal_kab_kota_original in ('IKN', 'KALIMANTAN SELATAN', 'BALIKPAPAN', 'KALIMANTAN TENGAH',
                     'SULAWESI SELATAN') then 112.7359184414066
            else asal_longitude
        end as asal_longitude,
        case
           when tujuan_kab_kota_original in ('BANDAR LAMPUNG', 'BENGKULU', 'JAMBI', 'LAMPUNG', 'RIAU', 'SUMATERA BARAT',
                     'SUMATERA SELATAN', 'SUMATERA UTARA', 'ACEH', 'BANGKA BELITUNG', 'KAB. LAMPUNG BARAT',
                     'KAB. LAMPUNG SELATAN', 'KAB. SIAK', 'KOTA MEDAN', 'KOTA PAGAR ALAM', 'KOTA PALEMBANG',
                     'KOTA PEKANBARU') then -5.933002183915104
            WHEN tujuan_kab_kota_original in ('BALI', 'KAB. BULELENG', 'NUSA TENGGARA BARAT', 'KAB. SUMBAWA', 'KOTA DENPASAR',
                     'KOTA MATARAM', 'NUSA TENGGARA BARAT', 'NUSA TENGGARA TIMUR') then -8.142900298290554
            WHEN tujuan_kab_kota_original in ('IKN', 'KALIMANTAN SELATAN', 'BALIKPAPAN', 'KALIMANTAN TENGAH',
                     'SULAWESI SELATAN') then -7.219475761110419
            else tujuan_latitude
        end as tujuan_latitude,
        case
            when tujuan_kab_kota_original in ('BANDAR LAMPUNG', 'BENGKULU', 'JAMBI', 'LAMPUNG', 'RIAU', 'SUMATERA BARAT',
                     'SUMATERA SELATAN', 'SUMATERA UTARA', 'ACEH', 'BANGKA BELITUNG', 'KAB. LAMPUNG BARAT',
                     'KAB. LAMPUNG SELATAN', 'KAB. SIAK', 'KOTA MEDAN', 'KOTA PAGAR ALAM', 'KOTA PALEMBANG',
                     'KOTA PEKANBARU') then 105.99945230749717
            WHEN tujuan_kab_kota_original in ('BALI', 'KAB. BULELENG', 'NUSA TENGGARA BARAT', 'KAB. SUMBAWA', 'KOTA DENPASAR',
                     'KOTA MATARAM', 'NUSA TENGGARA BARAT', 'NUSA TENGGARA TIMUR') then 114.40067867683435
            WHEN tujuan_kab_kota_original in ('IKN', 'KALIMANTAN SELATAN', 'BALIKPAPAN', 'KALIMANTAN TENGAH',
                     'SULAWESI SELATAN') then 112.7359184414066
            else tujuan_longitude
        end as tujuan_longitude
    from agg_data
    order by kode_titik, asal_tempat
    ;
"""
df = read_database(engine=engine, query=query)

# Export the DataFrame to a pickle file
pickle_file_path = 'data/origin_destination_agg.pkl'
df.to_pickle(pickle_file_path)
print(f"\nDataFrame exported to {pickle_file_path}")

df


DataFrame exported to data/origin_destination_agg.pkl


,kode_titik,asal_tempat,precentage_asal_tempat,tujuan_tempat,precentage_tujuan_tempat,asal_kab_kota,tujuan_kab_kota,_count_origin_dest_kab_kota,count_origin_dest_kab_kota,asal_latitude,asal_longitude,tujuan_latitude,tujuan_longitude
0,RSI1,Tempat Kerja,51.43,Tempat Kerja,47.37,BANDUNG,BANDUNG,2,10,-7.10000,107.60000,-7.100000,107.600000
1,RSI1,Tempat Kerja,51.43,Tempat Kerja,47.37,BANDUNG,EKSTERNAL,10,10,-7.10000,107.60000,-5.933002,105.999452
2,RSI1,Tempat Kerja,51.43,Tempat Kerja,47.37,BANDUNG BARAT,EKSTERNAL,4,10,-6.83333,107.48333,-5.933002,105.999452
3,RSI1,Tempat Kerja,51.43,Tempat Kerja,47.37,BANYUWANGI,EKSTERNAL,1,10,-8.33333,114.20000,-5.933002,105.999452
4,RSI1,Tempat Kerja,51.43,Tempat Kerja,47.37,BATANG,EKSTERNAL,1,10,-7.03333,109.88333,-5.933002,105.999452
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6293,RSI9,Tempat Usaha,8.15,Tempat Usaha,2.17,JEMBER,EKSTERNAL,35,40,-8.25000,113.65000,-8.142900,114.400679
6294,RSI9,Tempat Usaha,8.15,Tempat Usaha,2.17,KOTA SURABAYA,EKSTERNAL,36,40,-7.26667,112.71667,-8.142900,114.400679
6295,RSI9,Tempat Usaha,8.15,Tempat Usaha,2.17,KOTA SURABAYA,KOTA BANDUNG,1,10,-7.26667,112.71667,-6.917500,107.624440
6296,RSI9,Tempat Usaha,8.15,Tempat Usaha,2.17,PASURUAN,EKSTERNAL,9,10,-7.73333,112.83333,-8.142900,114.400679


In [3]:
survey_asal_tujuan_xlsx = "/mnt/d/PROJ UI/Data Primer Test/Data Primer Test/Survei Asal Tujuan/RSI JM Jawa 2024 - WorkFile - v511.xlsx"
survey_traffic_counting_jabo_xlsx = "/mnt/d/PROJ UI/Data Primer Test/Data Primer Test/Survei Traffic Counting/Rekapitulasi/Database Traffic Counting_JABO.xlsx"
survey_travel_journey_xlsx = "/mnt/d/PROJ UI/Data Primer Test/Data Primer Test/Survei Travel Journey/Travel Journeyy.xlsx"

# Survei Asal Tujuan

In [5]:
wb = openpyxl.load_workbook(survey_asal_tujuan_xlsx, data_only=True)
sheet_name = wb["FixUse"]  # Specify the sheet name

columns = [
    'kode_titik',
    'arah',
    'jenis',
    'trayek',
    'penumpang',
    'asal_tempat',
    'asal_tanggal',
    'asal_waktu',
    'asal_kab_kota',
    'asal_kec',
    'asal_kel',
    'penggunaan_tol',
    'tujuan_tempat',
    'tujuan_tanggal',
    'tujuan_waktu',
    'tujuan_kab_kota',
    'tujuan_kec',
    'tujuan_kel',
    'kode'
]

In [7]:
mapping_kode_titik = {}
sheet_readme = wb["README"]
for i in range(3, 12):
    mapping_kode_titik[str(sheet_readme[f'B{i}'].value).lower()] = str(sheet_readme[f'C{i}'].value)
    
mapping_kode_titik

{'rsi1': 'Pelabuhan Merak-Bakauheuni',
 'rsi2': 'Arteri Batas Jabar-Jateng (Utara)',
 'rsi3': 'Tol Batas Jabar-Jateng (Utara)',
 'rsi4': 'Arteri Batas Jabar-Jateng (Selatan)',
 'rsi5': 'Arteri Batas Jateng-Jatim (Utara)',
 'rsi6': 'Tol Batas Jateng-Jatim (Utara)',
 'rsi7': 'Arteri Batas Jateng-Jatim (Utara)',
 'rsi8': 'GT Cikampek-Kalihurip UTAMA',
 'rsi9': 'Pelabuhan Ketapang-Gilimanuk'}

In [8]:
data = []
for row in sheet_name.iter_rows(values_only=False, min_row=3):
    d_dict = {}
    if all(cell.value is None for cell in row):
        break
    
    for i in range(0, len(columns)):
        if row[i].is_date:
            d_dict[columns[i]] = row[i].number_format if row[i].value is None else str(row[i].value)
        else:
            d_dict[columns[i]]= row[i].value if row[i].value is not None else None
    d_dict["lokasi"] = mapping_kode_titik[d_dict["kode_titik"]]
    data.append(d_dict)

In [11]:
df_survey_asal_tujuan = pl.DataFrame(data)
print(df_survey_asal_tujuan)

# ddl_survey_asal_tujuan = generate_postgres_ddl(df=df_survey_asal_tujuan, table_name="survey_asal_tujuan")
# print(ddl_survey_asal_tujuan)
df_to_db(df=df_survey_asal_tujuan, table_name="survey_asal_tujuan")

shape: (18_888, 20)
┌────────────┬──────┬─────────┬────────┬───┬────────────┬────────────┬──────┬──────────────────────┐
│ kode_titik ┆ arah ┆ jenis   ┆ trayek ┆ … ┆ tujuan_kec ┆ tujuan_kel ┆ kode ┆ lokasi               │
│ ---        ┆ ---  ┆ ---     ┆ ---    ┆   ┆ ---        ┆ ---        ┆ ---  ┆ ---                  │
│ str        ┆ str  ┆ str     ┆ str    ┆   ┆ str        ┆ str        ┆ i64  ┆ str                  │
╞════════════╪══════╪═════════╪════════╪═══╪════════════╪════════════╪══════╪══════════════════════╡
│ rsi1       ┆ A    ┆ Truk 2G ┆        ┆ … ┆            ┆            ┆ 1    ┆ Pelabuhan            │
│            ┆      ┆         ┆        ┆   ┆            ┆            ┆      ┆ Merak-Bakauheuni     │
│ rsi1       ┆ A    ┆ Mobil   ┆        ┆ … ┆            ┆            ┆ 1    ┆ Pelabuhan            │
│            ┆      ┆         ┆        ┆   ┆            ┆            ┆      ┆ Merak-Bakauheuni     │
│ rsi6       ┆ B    ┆ Truk 3G ┆        ┆ … ┆            ┆            ┆ 

# Survei Traffic Counting

In [3]:
wb = openpyxl.load_workbook(survey_traffic_counting_jabo_xlsx, data_only=True)  # data_only=False keeps the formulas
sheet_name = wb["Data TC"]  # Specify the sheet name

columns = [
    'tc_code',
    'latitude',
    'longitude',
    'tahun',
    'bulan',
    'tanggal',
    'hari_kerja_hari_libur',
    'arah',
    'dari',
    'ke',
    'durasi',
    'waktu',
    'golongan_1',
    'golongan_2',
    'golongan_3',
    'golongan_4',
    'golongan_5',
    'golongan_6',
    'golongan_7',
    'golongan_8',
    'golongan_9',
    'golongan_10',
    'golongan_11',
    'golongan_12',
    'keterangan',
]

In [8]:
data = []
for row in sheet_name.iter_rows(values_only=True):
    d_dict = {}
    if all(cell is None for cell in row):
        break
    for i in range(0, len(columns)):
        d_dict[columns[i]] = row[i]
    data.append(d_dict)

In [ ]:
df_survey_traffic_counting_jabo = pl.DataFrame(data)
df_survey_traffic_counting_jabo = df_survey_traffic_counting_jabo.slice(1)

print(df_survey_traffic_counting_jabo)
ddl_survey_traffic_counting_jabo = generate_postgres_ddl(df=df_survey_traffic_counting_jabo, table_name="survey_traffic_counting_jabo")
print(ddl_survey_traffic_counting_jabo)
df_to_db(df=df_survey_traffic_counting_jabo, table_name="survey_traffic_counting_jabo")

# Survei Traffic Counting 2

In [72]:
survey_traffic_counting_folders = [
    "/mnt/d/PROJ UI/REV-4 (onedrive-FIN)/REV-4 (onedrive-FIN)/BDG",
    "/mnt/d/PROJ UI/REV-4 (onedrive-FIN)/REV-4 (onedrive-FIN)/JABO",
    "/mnt/d/PROJ UI/REV-4 (onedrive-FIN)/REV-4 (onedrive-FIN)/JAWA"
    ]

In [73]:
config_data = [
    [4, 2, "hari_tanggal"],
    [4, 12, "cuaca"],
    [5, 2, "nama_ruas_jalan"],
    [5, 8, "koordinat_lokasi"],
    [5, 12, "arah_dari"],
    [6, 2, "surveyor_rekam_hitung"],
    [6, 8, "durasi"],
    [6, 12, "arah_menuju"],
    [7, 2, "kode_lokasi"],
    [7, 12, "kode_arah"],
]

config_data_profil = [
    [6, 5, "catatan"],
    [3, 9, "jam_puncak_arah_1"],
    [4, 9, "vol_jam_puncak_arah_1"],
    [6, 9, "jam_puncak_arah_2"],
    [7, 9, "vol_jam_puncak_arah_2"],
]

table_data = {
    "i": [12, 203],
    "c": [1, 13],
    "mapping_col": {
        1: "rentang_survei",
        2: "Gol-6",
        3: "Gol-1-a",
        4: "Gol-1-b",
        5: "Gol-1-c",
        6: "Gol-1-d",
        7: "Gol-1-e",
        8: "Gol-2",
        9: "Gol-3",
        10: "Gol-4",
        11: "Gol-5",
        12: "total_tanpa_sepeda_motor",
        13: "total_dengan_sepeda_motor",
    }
}

data = []
for folder in survey_traffic_counting_folders:
    for filename in os.listdir(folder):
        f = os.path.join(folder, filename)
        print(filename)
        
        with open_workbook(f) as wb:
            profil_dict = {}
            for sheet_name in wb.sheets:
                if str(sheet_name).startswith(("Profil")):
                    with wb.get_sheet(sheet_name) as sheet:
                        for i, row in enumerate(sheet):
                            for c in range(len(row)):
                                for config in config_data_profil:
                                    if i == config[0] and c == config[1]:
                                        if str(sheet_name).endswith(("HK")):
                                            print(i, c, config[2])
                                            profil_dict[f"{config[2]}_hk"] = "%.2f" % row[c].v if isinstance(row[c].v, float) else row[c].v
                                        else:
                                            profil_dict[f"{config[2]}_hl"] = "%.2f" % row[c].v if isinstance(row[c].v, float) else row[c].v
                    
            for sheet_name in wb.sheets:
                if str(sheet_name).startswith(("HK_Arah", "HL_Arah")):
                    with wb.get_sheet(sheet_name) as sheet:
                        base_dict = {}
                        base_dict["filename"] = filename
                        base_dict["region"] = folder.split('/')[-1]
                        base_dict["sheet"] = sheet_name
                        for i, row in enumerate(sheet):
                            for c in range(len(row)):
                                if row[c].v:
                                    for config in config_data:
                                        if i == config[0] and c == config[1]:
                                            base_dict[config[2]] = int(row[c].v) if isinstance(row[c].v, float) else row[c].v
                        
                        d_dict = {}
                        for i, row in enumerate(sheet):
                            for c in range(len(row)):
                                if i >= table_data["i"][0] and i <= table_data["i"][1] and c >= table_data["c"][0] and c <= table_data["c"][1]:
                                    if c == 1:
                                        d_dict[table_data["mapping_col"][c]] = row[c].v if row[c].v else None
                                        # print(table_data["mapping_col"][c], row[c].v if row[c].v else None)
                                    else:
                                        d_dict[table_data["mapping_col"][c]] = int(row[c].v) if isinstance(row[c].v, float) else 0 
                                        # print(table_data["mapping_col"][c], int(row[c].v) if row[c].v else None)
                            if d_dict:
                                complete_dict = {**base_dict, **profil_dict, **d_dict}
                                data.append(complete_dict)
            print(len(data))
# df = pl.DataFrame(data)
# df["rentang_survei"].unique()
# df
            
# ddl = generate_postgres_ddl(df=df, table_name="survey_traffic_counting")    
# print(ddl)

ADDBDG-1.xlsb
3 9 jam_puncak_arah_1
4 9 vol_jam_puncak_arah_1
6 5 catatan
6 9 jam_puncak_arah_2
7 9 vol_jam_puncak_arah_2
1200
BDG-1.xlsb
3 9 jam_puncak_arah_1
4 9 vol_jam_puncak_arah_1
6 5 catatan
6 9 jam_puncak_arah_2
7 9 vol_jam_puncak_arah_2
2400
BDG-12.xlsb
3 9 jam_puncak_arah_1
4 9 vol_jam_puncak_arah_1
6 5 catatan
6 9 jam_puncak_arah_2
7 9 vol_jam_puncak_arah_2
3600
BDG-14.xlsb
3 9 jam_puncak_arah_1
4 9 vol_jam_puncak_arah_1
6 5 catatan
6 9 jam_puncak_arah_2
7 9 vol_jam_puncak_arah_2
4800
BDG-15.xlsb
3 9 jam_puncak_arah_1
4 9 vol_jam_puncak_arah_1
6 5 catatan
6 9 jam_puncak_arah_2
7 9 vol_jam_puncak_arah_2
6000
BDG-16.xlsb
3 9 jam_puncak_arah_1
4 9 vol_jam_puncak_arah_1
6 5 catatan
6 9 jam_puncak_arah_2
7 9 vol_jam_puncak_arah_2
7200
BDG-17.xlsb
3 9 jam_puncak_arah_1
4 9 vol_jam_puncak_arah_1
6 5 catatan
6 9 jam_puncak_arah_2
7 9 vol_jam_puncak_arah_2
8400
BDG-18.xlsb
3 9 jam_puncak_arah_1
4 9 vol_jam_puncak_arah_1
6 5 catatan
6 9 jam_puncak_arah_2
7 9 vol_jam_puncak_arah_2
9600

In [21]:
df = pl.DataFrame(data)
# df
# ddl = generate_postgres_ddl(df=df, table_name="survey_traffic_counting")    
# print(ddl)

df_to_db(df=df, table_name="survey_traffic_counting")

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "timestamp" of relation "survey_traffic_counting" does not exist
LINE 1: INSERT INTO survey_traffic_counting (timestamp, kmph, y, x, ...
                                             ^

[SQL: INSERT INTO survey_traffic_counting (timestamp, kmph, y, x, waktu, jarak, waktu_menit, jarak_km, arah, sheet, arah_awal, arah_akhir) VALUES (%(timestamp__0)s, %(kmph__0)s, %(y__0)s, %(x__0)s, %(waktu__0)s, %(jarak__0)s, %(waktu_menit__0)s, %(jarak_km ... 204468 characters truncated ... t__999)s, %(jarak_km__999)s, %(arah__999)s, %(sheet__999)s, %(arah_awal__999)s, %(arah_akhir__999)s)]
[parameters: {'jarak__0': None, 'timestamp__0': '08:38:07', 'arah_awal__0': 'Merak', 'waktu_menit__0': None, 'waktu__0': None, 'arah_akhir__0': 'Jatiasih', 'x__0': 105.999828, 'arah__0': 'arah A', 'sheet__0': 'Grafik TJ-1', 'kmph__0': 10, 'jarak_km__0': None, 'y__0': -5.93296, 'jarak__1': 4.455333882364654e-05, 'timestamp__1': '08:38:08', 'arah_awal__1': 'Mer

In [62]:

with open_workbook("/mnt/d/PROJ UI/REV-4 (onedrive-FIN)/REV-4 (onedrive-FIN)/BDG/ADDBDG-1.xlsb") as wb:
    for sheet_name in wb.sheets:
        if str(sheet_name).startswith(("Profil")):
            with wb.get_sheet("Profil_HK") as sheet:
                target_row = 5
                target_col = column_index_from_string("C")
                for i, row in enumerate(sheet):
                    row_data = []
                    for c in range(len(row)):
                        if row[c].v:
                            row_data.append({
                                "v": row[c].v,
                                "i": i,
                                "c": c
                                
                            })
                    print(row_data)

[{'v': 'ADDBDG-1_HK.xlsx', 'i': 0, 'c': 0}]
[]
[{'v': 'No./Kode Lokasi :', 'i': 2, 'c': 1}, {'v': 'ADDBDG-1', 'i': 2, 'c': 2}, {'v': 'Jumlah arah :', 'i': 2, 'c': 4}, {'v': 2.0, 'i': 2, 'c': 5}, {'v': 'Arah-1', 'i': 2, 'c': 8}]
[{'v': 'Durasi :', 'i': 3, 'c': 1}, {'v': '16', 'i': 3, 'c': 2}, {'v': 'jam', 'i': 3, 'c': 3}, {'v': 'Jumlah lajur per arah:', 'i': 3, 'c': 4}, {'v': '1', 'i': 3, 'c': 5}, {'v': 'Jam puncak :', 'i': 3, 'c': 8}, {'v': '17.00 - 18.00', 'i': 3, 'c': 9}, {'v': 'Ekivalen mobil penumpang', 'i': 3, 'c': 24}]
[{'v': 'Koordinat Lokasi :', 'i': 4, 'c': 1}, {'v': '-7.00150683, 107.7208633', 'i': 4, 'c': 2}, {'v': 'Terbagi (D) / tidak terbagi (UD) :', 'i': 4, 'c': 4}, {'v': 'UD', 'i': 4, 'c': 5}, {'v': 'Vol. jam puncak (smp/jam):', 'i': 4, 'c': 8}, {'v': 518.5, 'i': 4, 'c': 9}, {'v': 'MCY', 'i': 4, 'c': 24}, {'v': 'CAR', 'i': 4, 'c': 25}, {'v': 'LGV', 'i': 4, 'c': 26}, {'v': 'MGV', 'i': 4, 'c': 27}, {'v': 'HGV', 'i': 4, 'c': 28}, {'v': 'Angkot', 'i': 4, 'c': 29}, {'v': 'Bus

In [ ]:
config_data = [
    [4, 2, "hari_tanggal"],
    [4, 12, "cuaca"],
    [5, 2, "nama_ruas_jalan"],
    [5, 8, "koordinat_lokasi"],
    [5, 12, "arah_dari"],
    [6, 2, "surveyor_rekam_hitung"],
    [6, 8, "durasi"],
    [6, 12, "arah_menuju"],
    [7, 2, "kode_lokasi"],
    [7, 12, "kode_arah"],
]

table_data = {
    "i": [12, 203],
    "c": [1, 13],
    "mapping_col": {
        1: "rentang_survei",
        2: "Gol-6",
        3: "Gol-1-a",
        4: "Gol-1-b",
        5: "Gol-1-c",
        6: "Gol-1-d",
        7: "Gol-1-e",
        8: "Gol-2",
        9: "Gol-3",
        10: "Gol-4",
        11: "Gol-5",
        12: "total_tanpa_sepeda_motor",
        13: "total_dengan_sepeda_motor",
    }
}

config_data_profil = [
    [6, 5, "catatan"],
    [3, 9, "jam_puncak_arah_1"],
    [4, 9, "vol_jam_puncak_arah_1"],
    [6, 9, "jam_puncak_arah_2"],
    [7, 9, "vol_jam_puncak_arah_2"],
]
data = []


with open_workbook("/mnt/d/PROJ UI/REV-4 (onedrive-FIN)/REV-4 (onedrive-FIN)/JABO/JBO-22.xlsb") as wb:
    for sheet_name in wb.sheets:
        if str(sheet_name).startswith(("HK_Arah-1")):
            with wb.get_sheet(sheet_name) as sheet:
                base_dict = {}
                base_dict["filename"] = "filename"
                for i, row in enumerate(sheet):
                    for c in range(len(row)):
                        if row[c].v:
                            for config in config_data:
                                if i == config[0] and c == config[1]:
                                    base_dict[config[2]] = int(row[c].v) if isinstance(row[c].v, float) else row[c].v
                
                d_dict = {}
                for i, row in enumerate(sheet):
                    for c in range(len(row)):
                        if i >= table_data["i"][0] and i <= table_data["i"][1] and c >= table_data["c"][0] and c <= table_data["c"][1]:
                            if c == 1:
                                print(row[c].v)
                                d_dict[table_data["mapping_col"][c]] = row[c].v if row[c].v else None
                                # print(table_data["mapping_col"][c], row[c].v if row[c].v else None)
                            else:
                                print(table_data["mapping_col"][c], i, row[c].v)
                                d_dict[table_data["mapping_col"][c]] = int(row[c].v) if isinstance(row[c].v, float) else 0
                                # print(table_data["mapping_col"][c], int(row[c].v) if row[c].v else None)
                    if d_dict:
                        complete_dict = {**base_dict, **d_dict}
                        data.append(complete_dict)

# df = pl.DataFrame(data)
# # df["rentang_survei"].unique()
# # df
             
# ddl = generate_postgres_ddl(df=df, table_name="survey_traffic_counting")    
# print(ddl)                  

# Survei Travel Journey

In [4]:
wb = openpyxl.load_workbook(survey_travel_journey_xlsx, data_only=True)  # data_only=False keeps the formulas

columns = [
    'timestamp',
    'kmph',
    'y',
    'x',
    'waktu',
    'jarak',
    'waktu_menit',
    'jarak_km',
]

/home/delima/workpaces/test/proj-ui/venv/lib/python3.10/site-packages/openpyxl/reader/drawings.py:33: UserWarning: DrawingML support is incomplete and limited to charts and images only. Shapes and drawings will be lost.
  warn("DrawingML support is incomplete and limited to charts and images only. Shapes and drawings will be lost.")


In [4]:
sheet_base = wb['TJ-6']
arah_awal = sheet_base['C3'].value
arah_akhir = sheet_base['C4'].value

In [5]:
data = []
for config in sheet_config_travel_journey:
    for sheet_name in config["sheets"]:
        # if sheet_name == 'Grafik TJ-17.1':
        sheet_name_base = sheet_name.split(' ')[-1]
        sheet_base = wb[sheet_name_base]
        arah_awal = sheet_base['C3'].value
        arah_akhir = sheet_base['C4'].value
        gmap = sheet_base['D14'].value if not sheet_name_base == 'TJ-18' else sheet_base['D19'].value
        
        
        sheet = wb[sheet_name]
        for table in config["tables"]:
            # Convert column letters to column numbers
            start_col = column_index_from_string(table["start_col"])
            end_col = column_index_from_string(table["end_col"])
            print(table["row"], table["start_col"], table["end_col"])

            
            for row in sheet.iter_rows(min_row=table["row"], min_col=start_col, max_col=end_col, values_only=True):
                d_dict = {}
                if all(cell is None for cell in row):
                    break
                
                for i in range(0, len(columns)):
                    if columns[i] == 'kmph':
                        d_dict[columns[i]] = 0 if isinstance(row[i], datetime.datetime) else row[i]
                    else:
                        d_dict[columns[i]] = row[i] if not isinstance(row[i], datetime.time) else row[i].strftime("%H:%M:%S")
                    
                d_dict["arah"] = table["name"]
                d_dict["sheet"] = sheet_name
                d_dict["gmap"] = gmap
                if str(table["name"]).startswith("arah B"):
                    d_dict["arah_awal"] = arah_akhir
                    d_dict["arah_akhir"] = arah_awal
                else:
                    d_dict["arah_awal"] = arah_awal
                    d_dict["arah_akhir"] = arah_akhir
                data.append(d_dict)
            print(sheet_name, len(data))

49 B I
Grafik TJ-1 14912
49 K R
Grafik TJ-1 29851
49 B I
Grafik TJ-2 46450
49 K R
Grafik TJ-2 63158
49 B I
Grafik TJ-3 74109
49 K R
Grafik TJ-3 87846
49 B I
Grafik TJ-4 103404
49 K R
Grafik TJ-4 120162
49 B I
Grafik TJ-5 137204
49 K R
Grafik TJ-5 153235
49 B I
Grafik TJ-6 168497
49 K R
Grafik TJ-6 185515
49 B I
Grafik TJ-7 201182
49 K R
Grafik TJ-7 217264
49 B I
Grafik TJ-8 237486
49 K R
Grafik TJ-8 256131
49 B I
Grafik TJ-9 270232
49 K R
Grafik TJ-9 282919
49 B I
Grafik TJ-10.2 293034
49 K R
Grafik TJ-10.2 301538
49 B I
Grafik TJ-11.1 317399
49 K R
Grafik TJ-11.1 331541
49 B I
Grafik TJ-11.2 346213
49 K R
Grafik TJ-11.2 358803
49 B I
Grafik TJ-12.2 369327
49 K R
Grafik TJ-12.2 378689
49 B I
Grafik TJ-13.1 388659
49 K R
Grafik TJ-13.1 400462
49 B I
Grafik TJ-13.2 414065
49 K R
Grafik TJ-13.2 425879
49 B I
Grafik TJ-14 438810
49 K R
Grafik TJ-14 452978
49 B I
Grafik TJ-15.1 467736
49 K R
Grafik TJ-15.1 483417
47 B I
Grafik TJ-17.1 488635
47 K R
Grafik TJ-17.1 493879
47 T AA
Grafik TJ-17

In [7]:
df_survey_travel_journey = pl.DataFrame(data)
# print(df_survey_travel_journey)

# ddl_survey_travel_journey = generate_postgres_ddl(df=df_survey_travel_journey, table_name="survey_travel_journey")
# print(ddl_survey_travel_journey)
df_to_db(df=df_survey_travel_journey, table_name="survey_travel_journey")
# df_survey_travel_journey.filter(pl.col("arah") == "arah A - PM")

In [15]:
data = []
for config in sheet_config_travel_journey:
    for sheet_name in config["sheets"]:
        # if sheet_name == 'Grafik TJ-17.1':
        sheet_name_base = sheet_name.split(' ')[-1]
        
        sheet = wb[sheet_name_base]
        i = 1
        for table in config["tables_jarak"]:
            # Convert column letters to column numbers
            start_col = column_index_from_string(table["start_col"])
            end_col = column_index_from_string(table["end_col"])
            print(table["row"], table["start_col"], table["end_col"])

            
            for row in sheet.iter_rows(min_row=table["row"], min_col=start_col, max_col=end_col, values_only=True):
                d_dict = {}
                if all(cell is None for cell in row):
                    break
                
                d_dict["jarak"] = row[0]
                
                d_dict["arah"] = table["name"]
                d_dict["sheet"] = sheet_name
                if str(table["name"]).startswith("arah B"):
                    i -= 1
                    d_dict["seq"] = i
                else:
                    d_dict["seq"] = i
                    i += 1
                data.append(d_dict)
            print(sheet_name, len(data))

7 D D
Grafik TJ-1 5
7 H H
Grafik TJ-1 10
7 D D
Grafik TJ-2 15
7 H H
Grafik TJ-2 20
7 D D
Grafik TJ-3 25
7 H H
Grafik TJ-3 30
7 D D
Grafik TJ-4 35
7 H H
Grafik TJ-4 40
7 D D
Grafik TJ-5 45
7 H H
Grafik TJ-5 50
7 D D
Grafik TJ-6 55
7 H H
Grafik TJ-6 60
7 D D
Grafik TJ-7 65
7 H H
Grafik TJ-7 70
7 D D
Grafik TJ-8 75
7 H H
Grafik TJ-8 80
7 D D
Grafik TJ-9 85
7 H H
Grafik TJ-9 90
7 D D
Grafik TJ-10.2 95
7 H H
Grafik TJ-10.2 100
7 D D
Grafik TJ-11.1 105
7 H H
Grafik TJ-11.1 110
7 D D
Grafik TJ-11.2 115
7 H H
Grafik TJ-11.2 120
7 D D
Grafik TJ-12.2 125
7 H H
Grafik TJ-12.2 130
7 D D
Grafik TJ-13.1 135
7 H H
Grafik TJ-13.1 140
7 D D
Grafik TJ-13.2 145
7 H H
Grafik TJ-13.2 150
7 D D
Grafik TJ-14 155
7 H H
Grafik TJ-14 160
7 D D
Grafik TJ-15.1 165
7 H H
Grafik TJ-15.1 170
7 D D
Grafik TJ-17.1 175
7 H H
Grafik TJ-17.1 180
7 D D
Grafik TJ-17.2 185
7 H H
Grafik TJ-17.2 190
7 D D
Grafik TJ-17.3 195
7 H H
Grafik TJ-17.3 200
7 D D
Grafik TJ-18 210
7 H H
Grafik TJ-18 220


In [20]:
df_survey_travel_journey_jarak = pl.DataFrame(data)

# ddl_survey_travel_journey_jarak = generate_postgres_ddl(df=df_survey_travel_journey_jarak, table_name="survey_travel_journey_jarak")
# print(ddl_survey_travel_journey_jarak)
df_to_db(df=df_survey_travel_journey_jarak, table_name="survey_travel_journey_jarak")

In [1]:
from config import sheet_config_travel_journey
sheet_config_travel_journey

[{'sheets': ['Grafik TJ-17.1'],
  'tables': [{'name': 'arah A - AM',
    'row': 47,
    'start_col': 'B',
    'end_col': 'I'},
   {'name': 'arah B - AM', 'row': 47, 'start_col': 'K', 'end_col': 'R'},
   {'name': 'arah A - PM', 'row': 47, 'start_col': 'T', 'end_col': 'AA'},
   {'name': 'arah B - PM', 'row': 47, 'start_col': 'AC', 'end_col': 'AJ'}]}]

In [2]:
import folium
folium.__version__

'0.17.0'

In [3]:
m = folium.Map(location=(30, 20), zoom_start=4)

def style_function(feature):
    area = int(feature['properties']['area'])
    return {
        'fillOpacity': 0.5,
        'weight': 0.5,
        'fillColor': 'green' if area < 10008018378 \
               else 'orange'if area< 20008018378\
                else 'red'
    }

gjson = folium.GeoJson("https://raw.githubusercontent.com/stefanocudini/leaflet-geojson-selector/master/examples/italy-regions.json", 
                       style_function=style_function
                      ).add_to(m)

In [42]:
from utils.util import engine, read_database
query = """
    select
        kode_lokasi,
        sheet,
        FLOOR(cast(trim(split_part(rentang_survei, '-', 1)) as float)) as start_rentang_survei,
        (FLOOR(cast(trim(split_part(rentang_survei, '-', 1)) as float)):: varchar) || '.00 - ' || (CEIL(cast(trim(split_part(rentang_survei, '-', 2)) as float)):: varchar) || '.00' as rentang_survei_,
        sum("Gol-6") as gol_6,
        sum("Gol-1-a") as gol_1_a,
        sum("Gol-1-b") as gol_1_b,
        sum("Gol-1-c") as gol_1_c,
        sum("Gol-1-d") as gol_1_d,
        sum("Gol-1-a") as gol_1_e,
        sum("Gol-2") as gol_2,
        sum("Gol-3") as gol_3,
        sum("Gol-4") as gol_4,
        sum("Gol-5") as gol_5,
        sum(total_tanpa_sepeda_motor) as total_tanpa_sepeda_motor,
        sum(total_dengan_sepeda_motor) as total_dengan_sepeda_motor
    from
        survey_traffic_counting
    where kode_lokasi = 'ADDJBO-1'
    group by
        kode_lokasi,
        sheet,
        start_rentang_survei,
        rentang_survei_
"""
df = read_database(engine=engine, query=query)
df

,kode_lokasi,sheet,start_rentang_survei,rentang_survei_,gol_6,gol_1_a,gol_1_b,gol_1_c,gol_1_d,gol_1_e,gol_2,gol_3,gol_4,gol_5,total_tanpa_sepeda_motor,total_dengan_sepeda_motor
0,ADDJBO-1,HK_Arah-1,0.0,0.00 - 1.00,174,43,0,0,0,43,7,42,6,2,110,284
1,ADDJBO-1,HK_Arah-1,1.0,1.00 - 2.00,78,17,0,0,0,17,13,43,3,0,84,162
2,ADDJBO-1,HK_Arah-1,2.0,2.00 - 3.00,63,20,1,0,0,20,9,67,3,4,113,176
3,ADDJBO-1,HK_Arah-1,3.0,3.00 - 4.00,14,17,0,0,1,17,12,63,6,1,106,120
4,ADDJBO-1,HK_Arah-1,4.0,4.00 - 5.00,33,10,0,0,1,10,13,55,10,0,95,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,ADDJBO-1,HL_Arah-2,19.0,19.00 - 20.00,910,163,4,1,0,163,15,3,3,1,200,1110
92,ADDJBO-1,HL_Arah-2,20.0,20.00 - 21.00,789,150,2,1,0,150,15,20,5,1,204,993
93,ADDJBO-1,HL_Arah-2,21.0,21.00 - 22.00,742,127,1,0,4,127,11,14,7,3,177,919
94,ADDJBO-1,HL_Arah-2,22.0,22.00 - 23.00,524,111,1,0,1,111,9,34,1,4,169,693


In [44]:
df_stats_hk_jabo = df.loc[(df['kode_lokasi'] == 'ADDJBO-1') & (df['sheet'].str.startswith("HK"))]
# df_stats_hk_jabo
df_stats_hk_jabo = df_stats_hk_jabo.groupby('rentang_survei_', as_index=False).sum()
df_stats_hk_jabo = df_stats_hk_jabo.sort_values(by='start_rentang_survei').drop(columns=['start_rentang_survei', 'kode_lokasi', 'sheet'])
df_stats_hk_jabo

,rentang_survei_,gol_6,gol_1_a,gol_1_b,gol_1_c,gol_1_d,gol_1_e,gol_2,gol_3,gol_4,gol_5,total_tanpa_sepeda_motor,total_dengan_sepeda_motor
0,0.00 - 1.00,290,66,0,0,3,66,13,182,15,6,298,588
1,1.00 - 2.00,129,32,0,0,0,32,15,165,7,2,231,360
12,2.00 - 3.00,99,35,1,0,0,35,15,133,5,4,206,305
17,3.00 - 4.00,39,25,1,0,1,25,15,117,11,1,181,220
18,4.00 - 5.00,89,35,0,0,2,35,19,92,18,0,175,264
19,5.00 - 6.00,504,119,12,4,6,119,25,48,16,1,243,747
20,6.00 - 7.00,2935,316,53,2,1,316,19,7,6,2,422,3357
21,7.00 - 8.00,3012,299,48,1,1,299,23,8,1,0,413,3425
22,8.00 - 9.00,2070,286,34,0,1,286,68,25,8,4,486,2556
23,9.00 - 10.00,1680,275,28,1,0,275,109,33,13,6,556,2236


In [16]:
query = """
with base_data as (
    select
        row_number() over (partition by sheet, arah order by coalesce(jarak, 0.0) asc) as rn,
        count(*) over (partition by sheet, arah) as count_per_sheet_arah,
        *
    from survey_travel_journey
    where sheet = 'Grafik TJ-1'
        and arah = 'arah A'
)
select
    case
        when rn = 1 then true
        when rn = ((1.0/4.0) * count_per_sheet_arah)::int then true
        when rn = ((2.0/4.0) * count_per_sheet_arah)::int then true
        when rn = ((3.0/4.0) * count_per_sheet_arah)::int then true
        when rn = count_per_sheet_arah then true
        else false
    END as flag_point,
    *
from
    base_data
;

"""

df = read_database(engine=engine, query=query)
df

,flag_point,rn,count_per_sheet_arah,timestamp,kmph,y,x,waktu,jarak,waktu_menit,jarak_km,arah,sheet,arah_awal,arah_akhir
0,True,1,14912,08:38:07,10,-5.932960,105.999828,None,NaN,NaN,NaN,arah A,Grafik TJ-1,Merak,Jatiasih
1,False,2,14912,08:38:08,17,-5.932991,105.999860,00:00:01,0.000045,0.016667,0.004967,arah A,Grafik TJ-1,Merak,Jatiasih
2,False,3,14912,08:38:09,24,-5.933032,105.999903,00:00:02,0.000104,0.033333,0.011590,arah A,Grafik TJ-1,Merak,Jatiasih
3,False,4,14912,08:38:10,31,-5.933083,105.999957,00:00:03,0.000178,0.050000,0.019870,arah A,Grafik TJ-1,Merak,Jatiasih
4,False,5,14912,08:38:11,36,-5.933141,106.000023,00:00:04,0.000266,0.066667,0.029665,arah A,Grafik TJ-1,Merak,Jatiasih
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14907,False,14908,14912,14:04:15,10,-6.292011,106.958128,05:26:08,1.339121,326.133333,149.281995,arah A,Grafik TJ-1,Merak,Jatiasih
14908,False,14909,14912,14:04:16,11,-6.292034,106.958158,05:26:09,1.339158,326.150000,149.286209,arah A,Grafik TJ-1,Merak,Jatiasih
14909,False,14910,14912,14:04:17,12,-6.292062,106.958185,05:26:10,1.339197,326.166667,149.290545,arah A,Grafik TJ-1,Merak,Jatiasih
14910,False,14911,14912,14:04:18,13,-6.292095,106.958211,05:26:11,1.339239,326.183333,149.295229,arah A,Grafik TJ-1,Merak,Jatiasih


In [17]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# Initialize Nominatim API for reverse geocoding
geolocator = Nominatim(user_agent="streamlit-geocoder")

def get_city_name(lat, lon):
    """Get city name from latitude and longitude using reverse geocoding."""
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        print(location.raw['address'])
        if location and 'city' in location.raw['address']:
            return location.raw['address']['city']
        elif location and 'town' in location.raw['address']:
            return location.raw['address']['town']
        elif location and 'village' in location.raw['address']:
            return location.raw['address']['village']
        else:
            return "City not found"
    except GeocoderTimedOut:
        return "Request timed out"

# Example list of coordinates
coordinates = [
    {"lat": -5.93296, "lon": 105.999828},  # Example coordinate for Merak
    {"lat": -6.1104, "lon": 106.1496},  # Example coordinate for Serang
    {"lat": -6.2653, "lon": 106.4206},  # Example coordinate for Balaraja
    {"lat": -6.1787, "lon": 106.7958},  # Example coordinate for Tomang (JORR)
    {"lat": -6.3166, "lon": 106.9894}   # Example coordinate for Jatiasih (JORR)
]


# Display coordinates and city names in Streamlit
# for i, coord in enumerate(coordinates, 1):
#     city_name = get_city_name(coord["lat"], coord["lon"])
    
for idx, row in df.iterrows():
    if row["flag_point"] is True:
        # print(row["y"], row["x"])
        print(row["rn"], row["y"], row["x"])
        get_city_name(row["y"], row["x"])

1 -5.93296 105.999828
{'road': 'Jalan Yos Sudarso', 'town': 'Merak', 'city': 'Cilegon', 'region': 'Jawa', 'ISO3166-2-lvl3': 'ID-JW', 'postcode': '42438', 'country': 'Indonesia', 'country_code': 'id'}
3728 -6.119881 106.199093
{'road': '1', 'village': 'Panancangan', 'district': 'Cipocok Jaya', 'city': 'Serang', 'region': 'Jawa', 'ISO3166-2-lvl3': 'ID-JW', 'postcode': '42183', 'country': 'Indonesia', 'country_code': 'id'}
7456 -6.198953 106.456319
{'road': 'Jalan Raya Serang', 'town': 'Balaraja', 'municipality': 'Balaraja', 'county': 'Kabupaten Tangerang', 'region': 'Jawa', 'ISO3166-2-lvl3': 'ID-JW', 'postcode': '15610', 'country': 'Indonesia', 'country_code': 'id'}
11184 -6.164544 106.653621
{'road': 'Jalan Lio Baru', 'suburb': 'Batusari', 'city_district': 'Batuceper', 'city': 'Tangerang', 'state': 'Banten', 'ISO3166-2-lvl4': 'ID-BT', 'region': 'Jawa', 'ISO3166-2-lvl3': 'ID-JW', 'postcode': '15119', 'country': 'Indonesia', 'country_code': 'id'}
14912 -6.292129 106.958237
{'road': 'Jalan

In [25]:
from utils.util import engine, read_database

query = """
with base_data as (
    select
        row_number() over (partition by sheet, arah order by coalesce(jarak, 0.0) asc) as rn,
        count(*) over (partition by sheet, arah) as count_per_sheet_arah,
        *
    from survey_travel_journey
    where sheet in ('Grafik TJ-1', 'Grafik TJ-11.1')
        and arah = 'arah A'
)
select
    case
        when rn = 1 then 1
        when rn = ((1.0/4.0) * count_per_sheet_arah)::int then 2
        when rn = ((2.0/4.0) * count_per_sheet_arah)::int then 3
        when rn = ((3.0/4.0) * count_per_sheet_arah)::int then 4
        when rn = count_per_sheet_arah then 5
    END as flag_point,
    *
from
    base_data
;

"""

df = read_database(engine=engine, query=query)
df = df.sort_values(by='rn')

In [26]:
coordinates = []
for idx, row in df.iterrows():
    coordinates.append([row["x"], row["y"]])

geojson_template = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "LineString",
        "coordinates": coordinates
      },
      "properties": {
        "name": "Sample Route",
        "description": "A sample route for testing purposes."
      }
    }
  ]
}



In [27]:
import json
# Specify the file path to save the GeoJSON file
file_path = "sample_route.geojson"

# Write the dictionary to a .geojson file
with open(file_path, "w") as geojson_file:
    json.dump(geojson_template, geojson_file, indent=4)

print(f"GeoJSON data saved to {file_path}")

GeoJSON data saved to sample_route.geojson


In [4]:
import json

# Open the JSON file
with open('/mnt/c/Users/097971/Downloads/provinces.json', 'r') as file:
    # Load the content as a Python list of dictionaries
    data = json.load(file)

# Print the result to check
print(data)

# Confirm the type
# print(type(data))
df = pl.DataFrame(data)
# df_survey_traffic_counting_jabo = df_survey_traffic_counting_jabo.slice(1)

# print(df_survey_traffic_counting_jabo)
# ddl_survey_traffic_counting_jabo = generate_postgres_ddl(df=df_survey_traffic_counting_jabo, table_name="survey_traffic_counting_jabo")
# print(ddl_survey_traffic_counting_jabo)
df_to_db(df=df, table_name="provinces")
# df


[{'id': '11', 'name': 'ACEH', 'alt_name': 'ACEH', 'latitude': 4.36855, 'longitude': 97.0253}, {'id': '12', 'name': 'SUMATERA UTARA', 'alt_name': 'SUMATERA UTARA', 'latitude': 2.19235, 'longitude': 99.38122}, {'id': '13', 'name': 'SUMATERA BARAT', 'alt_name': 'SUMATERA BARAT', 'latitude': -1.34225, 'longitude': 100.0761}, {'id': '14', 'name': 'RIAU', 'alt_name': 'RIAU', 'latitude': 0.50041, 'longitude': 101.54758}, {'id': '15', 'name': 'JAMBI', 'alt_name': 'JAMBI', 'latitude': -1.61157, 'longitude': 102.7797}, {'id': '16', 'name': 'SUMATERA SELATAN', 'alt_name': 'SUMATERA SELATAN', 'latitude': -3.12668, 'longitude': 104.09306}, {'id': '17', 'name': 'BENGKULU', 'alt_name': 'BENGKULU', 'latitude': -3.51868, 'longitude': 102.53598}, {'id': '18', 'name': 'LAMPUNG', 'alt_name': 'LAMPUNG', 'latitude': -4.8555, 'longitude': 105.0273}, {'id': '19', 'name': 'KEPULAUAN BANGKA BELITUNG', 'alt_name': 'KEPULAUAN BANGKA BELITUNG', 'latitude': -2.75775, 'longitude': 107.58394}, {'id': '21', 'name': 'K

In [19]:
data = pd.DataFrame({
    'origin_lat': [-6.2, -7.0, -7.0],
    'origin_lon': [106.8, 110.4, 110.4],
    'dest_lat': [-7.8, -6.9, -6.9],
    'dest_lon': [112.5, 113.9, -6.9],
    'weight': [50, 80, 70]  # Line thickness or color indicator
})

data

,origin_lat,origin_lon,dest_lat,dest_lon,weight
0,-6.2,106.8,-7.8,112.5,50
1,-7.0,110.4,-6.9,113.9,80
2,-7.0,110.4,-6.9,-6.9,70


In [13]:
query = """
        with
        grouping_data as (
            select
                kode_titik,
                case
                    when asal_tempat in ('-', '0') then 'Tempat Tidak Diketahui'
                    when asal_tempat in ('tempatkerja') then 'Tempat Kerja'
                    when asal_tempat in ('tempatpendidikan') then 'Tempat Pendidikan'
                    when asal_tempat in ('tempatperbelanjaan') then 'Tempat Perbelanjaan'
                    when asal_tempat in ('tempatrekreasi') then 'Tempat Rekreasi'
                    when asal_tempat in ('tempattinggal') then 'Tempat Tinggal'
                    when asal_tempat in ('tempatusaha') then 'Tempat Usaha'
                end as asal_tempat,
                case
                    when tujuan_tempat in ('-', '0') then 'Tempat Tidak Diketahui'
                    when tujuan_tempat in ('tempatkerja') then 'Tempat Kerja'
                    when tujuan_tempat in ('tempatpendidikan') then 'Tempat Pendidikan'
                    when tujuan_tempat in ('tempatperbelanjaan') then 'Tempat Perbelanjaan'
                    when tujuan_tempat in ('tempatrekreasi') then 'Tempat Rekreasi'
                    when tujuan_tempat in ('tempattinggal') then 'Tempat Tinggal'
                    when tujuan_tempat in ('tempatusaha') then 'Tempat Usaha'
                end as tujuan_tempat,
                upper(trim(asal_kab_kota)) as asal_kab_kota_original,
                upper(trim(tujuan_kab_kota)) as tujuan_kab_kota_original,
                case
                    when upper(trim(asal_kab_kota)) in ('BALI', 'BANDAR LAMPUNG', 'BENGKULU', 'JAMBI', 'LAMPUNG', 'RIAU',
                                        'SUMATERA BARAT', 'SUMATERA SELATAN', 'SUMATERA UTARA', 'KALIMANTAN TENGAH',
                                        'SULAWESI SELATAN', 'BALIKPAPAN', 'NUSA TENGGARA BARAT', 'KALIMANTAN SELATAN',
                                        'KALIMANTAN TENGAH', 'KAB. BULELENG') then 'EKSTERNAL'
                    else REPLACE(asal_kab_kota, 'KAB. ', '')
                end as asal_kab_kota,
                case
                when upper(trim(tujuan_kab_kota)) in ('BALI', 'BANDAR LAMPUNG', 'BENGKULU', 'JAMBI', 'LAMPUNG', 'RIAU',
                                    'SUMATERA BARAT', 'SUMATERA SELATAN', 'SUMATERA UTARA', 'KALIMANTAN TENGAH',
                                    'SULAWESI SELATAN', 'BALIKPAPAN', 'NUSA TENGGARA BARAT', 'KALIMANTAN SELATAN',
                                    'KALIMANTAN TENGAH', 'ACEH', 'KOTA MEDAN', 'IKN', 'KOTA PALEMBANG',
                                    'BANGKA BELITUNG', 'KOTA PEKANBARU', 'NUSA TENGGARA TIMUR', 'KAB. LAMPUNG SELATAN',
                                    'KAB. LAMPUNG BARAT', 'KOTA DENPASAR', 'KOTA MATARAM', 'KAB. SUMBAWA', 'KAB. SIAK',
                                    'KOTA PAGAR ALAM') then 'EKSTERNAL'
                else REPLACE(tujuan_kab_kota, 'KAB. ', '')
                end as tujuan_kab_kota,
                r_asal.latitude as asal_latitude,
                r_asal.longitude as asal_longitude,
                r_tujuan.latitude as tujuan_latitude,
                r_tujuan.longitude as tujuan_longitude
            from survey_asal_tujuan s
            left join regencies r_asal
            on REPLACE(upper(REPLACE(upper(s.asal_kab_kota), 'ADM. ', '')), 'KAB. ', 'KABUPATEN ') = r_asal.name
            left join regencies r_tujuan
            on REPLACE(upper(REPLACE(upper(s.tujuan_kab_kota), 'ADM. ', '')), 'KAB. ', 'KABUPATEN ') = r_tujuan.name
        ),
        agg_data as (
            select
                upper(kode_titik) as kode_titik,
                COUNT(1) OVER (PARTITION BY kode_titik) AS count_data,
                COUNT(1) OVER (PARTITION BY kode_titik, asal_tempat) AS count_origin,
                COUNT(1) OVER (PARTITION BY kode_titik, tujuan_tempat) AS count_dest,
                COUNT(1) OVER (PARTITION BY kode_titik, asal_tempat, tujuan_tempat) AS count_origin_dest,
                COUNT(1) OVER (PARTITION BY kode_titik, asal_kab_kota, tujuan_kab_kota) AS count_origin_dest_kab_kota,
                asal_tempat,
                tujuan_tempat,
                asal_kab_kota,
                asal_kab_kota_original,
                tujuan_kab_kota,
                tujuan_kab_kota_original,
                asal_latitude,
                asal_longitude,
                tujuan_latitude,
                tujuan_longitude
        from grouping_data
    )

    select
        distinct
        upper(kode_titik) as kode_titik,
        asal_tempat,
        ROUND(((count_origin::float / count_data::float) * 100)::numeric, 2) as precentage_asal_tempat,
        tujuan_tempat,
        ROUND(((count_dest::float / count_data::float) * 100)::numeric, 2) as precentage_tujuan_tempat,
        asal_kab_kota,
        asal_kab_kota_original,
        tujuan_kab_kota,
        count_origin_dest_kab_kota as _count_origin_dest_kab_kota,
        case
            when count_origin_dest_kab_kota <= 10 then 10
            when count_origin_dest_kab_kota > 10 and count_origin_dest_kab_kota <= 20 then 20
            when count_origin_dest_kab_kota > 20 and count_origin_dest_kab_kota <= 30 then 30
            when count_origin_dest_kab_kota > 30 and count_origin_dest_kab_kota <= 40 then 40
            when count_origin_dest_kab_kota > 40 and count_origin_dest_kab_kota <= 50 then 50
            when count_origin_dest_kab_kota > 50 and count_origin_dest_kab_kota <= 60 then 60
            when count_origin_dest_kab_kota > 60 and count_origin_dest_kab_kota <= 70 then 70
            when count_origin_dest_kab_kota > 70 and count_origin_dest_kab_kota <= 80 then 80
            when count_origin_dest_kab_kota > 80 and count_origin_dest_kab_kota <= 90 then 90
            when count_origin_dest_kab_kota > 90 and count_origin_dest_kab_kota <= 100 then 100
            when count_origin_dest_kab_kota > 100 and count_origin_dest_kab_kota <= 500 then 500
            when count_origin_dest_kab_kota > 500 and count_origin_dest_kab_kota <= 1000 then 1000
            when count_origin_dest_kab_kota > 1000 and count_origin_dest_kab_kota <= 1500 then 1500
            when count_origin_dest_kab_kota > 1500 and count_origin_dest_kab_kota <= 2000 then 2000
        end count_origin_dest_kab_kota,
        case
            when asal_kab_kota_original in ('BANDAR LAMPUNG', 'BENGKULU', 'JAMBI', 'LAMPUNG', 'RIAU', 'SUMATERA BARAT',
                     'SUMATERA SELATAN', 'SUMATERA UTARA', 'ACEH', 'BANGKA BELITUNG', 'KAB. LAMPUNG BARAT',
                     'KAB. LAMPUNG SELATAN', 'KAB. SIAK', 'KOTA MEDAN', 'KOTA PAGAR ALAM', 'KOTA PALEMBANG',
                     'KOTA PEKANBARU') then -5.933002183915104
            WHEN asal_kab_kota_original in ('BALI', 'KAB. BULELENG', 'NUSA TENGGARA BARAT', 'KAB. SUMBAWA', 'KOTA DENPASAR',
                     'KOTA MATARAM', 'NUSA TENGGARA BARAT', 'NUSA TENGGARA TIMUR') then -8.142900298290554
            WHEN asal_kab_kota_original in ('IKN', 'KALIMANTAN SELATAN', 'BALIKPAPAN', 'KALIMANTAN TENGAH',
                     'SULAWESI SELATAN') then -7.219475761110419
            else asal_latitude
        end as asal_latitude,
        case
            when asal_kab_kota_original in ('BANDAR LAMPUNG', 'BENGKULU', 'JAMBI', 'LAMPUNG', 'RIAU', 'SUMATERA BARAT',
                     'SUMATERA SELATAN', 'SUMATERA UTARA', 'ACEH', 'BANGKA BELITUNG', 'KAB. LAMPUNG BARAT',
                     'KAB. LAMPUNG SELATAN', 'KAB. SIAK', 'KOTA MEDAN', 'KOTA PAGAR ALAM', 'KOTA PALEMBANG',
                     'KOTA PEKANBARU') then 105.99945230749717
            WHEN asal_kab_kota_original in ('BALI', 'KAB. BULELENG', 'NUSA TENGGARA BARAT', 'KAB. SUMBAWA', 'KOTA DENPASAR',
                     'KOTA MATARAM', 'NUSA TENGGARA BARAT', 'NUSA TENGGARA TIMUR') then 114.40067867683435
            WHEN asal_kab_kota_original in ('IKN', 'KALIMANTAN SELATAN', 'BALIKPAPAN', 'KALIMANTAN TENGAH',
                     'SULAWESI SELATAN') then 112.7359184414066
            else asal_longitude
        end as asal_longitude,
        case
           when tujuan_kab_kota_original in ('BANDAR LAMPUNG', 'BENGKULU', 'JAMBI', 'LAMPUNG', 'RIAU', 'SUMATERA BARAT',
                     'SUMATERA SELATAN', 'SUMATERA UTARA', 'ACEH', 'BANGKA BELITUNG', 'KAB. LAMPUNG BARAT',
                     'KAB. LAMPUNG SELATAN', 'KAB. SIAK', 'KOTA MEDAN', 'KOTA PAGAR ALAM', 'KOTA PALEMBANG',
                     'KOTA PEKANBARU') then -5.933002183915104
            WHEN tujuan_kab_kota_original in ('BALI', 'KAB. BULELENG', 'NUSA TENGGARA BARAT', 'KAB. SUMBAWA', 'KOTA DENPASAR',
                     'KOTA MATARAM', 'NUSA TENGGARA BARAT', 'NUSA TENGGARA TIMUR') then -8.142900298290554
            WHEN tujuan_kab_kota_original in ('IKN', 'KALIMANTAN SELATAN', 'BALIKPAPAN', 'KALIMANTAN TENGAH',
                     'SULAWESI SELATAN') then -7.219475761110419
            else tujuan_latitude
        end as tujuan_latitude,
        case
            when tujuan_kab_kota_original in ('BANDAR LAMPUNG', 'BENGKULU', 'JAMBI', 'LAMPUNG', 'RIAU', 'SUMATERA BARAT',
                     'SUMATERA SELATAN', 'SUMATERA UTARA', 'ACEH', 'BANGKA BELITUNG', 'KAB. LAMPUNG BARAT',
                     'KAB. LAMPUNG SELATAN', 'KAB. SIAK', 'KOTA MEDAN', 'KOTA PAGAR ALAM', 'KOTA PALEMBANG',
                     'KOTA PEKANBARU') then 105.99945230749717
            WHEN tujuan_kab_kota_original in ('BALI', 'KAB. BULELENG', 'NUSA TENGGARA BARAT', 'KAB. SUMBAWA', 'KOTA DENPASAR',
                     'KOTA MATARAM', 'NUSA TENGGARA BARAT', 'NUSA TENGGARA TIMUR') then 114.40067867683435
            WHEN tujuan_kab_kota_original in ('IKN', 'KALIMANTAN SELATAN', 'BALIKPAPAN', 'KALIMANTAN TENGAH',
                     'SULAWESI SELATAN') then 112.7359184414066
            else tujuan_longitude
        end as tujuan_longitude
    from agg_data
    order by kode_titik, asal_tempat
    ; 
"""
df_agg = read_database(engine=engine, query=query)


In [16]:
df_agg = df_agg.loc[df_agg['kode_titik'] == "RSI2"]
df_agg

,kode_titik,asal_tempat,precentage_asal_tempat,tujuan_tempat,precentage_tujuan_tempat,asal_kab_kota,asal_kab_kota_original,tujuan_kab_kota,_count_origin_dest_kab_kota,count_origin_dest_kab_kota,asal_latitude,asal_longitude,tujuan_latitude,tujuan_longitude
419,RSI2,Tempat Kerja,40.65,Tempat Kerja,42.10,BANDUNG,KAB. BANDUNG,BREBES,5,10,-7.10000,107.60000,-7.05000,108.90000
420,RSI2,Tempat Kerja,40.65,Tempat Kerja,42.10,BANDUNG,KAB. BANDUNG,KOTA SURABAYA,3,10,-7.10000,107.60000,-7.26667,112.71667
421,RSI2,Tempat Kerja,40.65,Tempat Kerja,42.10,BANDUNG,KAB. BANDUNG,KOTA TEGAL,1,10,-7.10000,107.60000,-6.86860,109.11290
422,RSI2,Tempat Kerja,40.65,Tempat Kerja,42.10,BANDUNG,KAB. BANDUNG,KOTA YOGYAKARTA,2,10,-7.10000,107.60000,-7.80000,110.37500
423,RSI2,Tempat Kerja,40.65,Tempat Kerja,42.10,BANDUNG BARAT,KAB. BANDUNG BARAT,PATI,1,10,-6.83333,107.48333,-6.76667,111.10000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1360,RSI2,Tempat Usaha,3.33,Tempat Usaha,7.04,MALANG,KAB. MALANG,BEKASI,2,10,-8.16667,112.66667,-6.24667,107.10833
1361,RSI2,Tempat Usaha,3.33,Tempat Usaha,7.04,SERANG,KAB. SERANG,KOTA SURABAYA,1,10,-6.15000,106.00000,-7.26667,112.71667
1362,RSI2,Tempat Usaha,3.33,Tempat Usaha,7.04,SIDOARJO,KAB. SIDOARJO,TANGERANG,1,10,-7.45000,112.70000,-6.20000,106.46667
1363,RSI2,Tempat Usaha,3.33,Tempat Usaha,7.04,TEGAL,KAB. TEGAL,CIREBON,49,50,-7.03333,109.16667,-6.80000,108.56667


In [104]:
import pandas as pd
import numpy as np
import plotly.express as px

# Example dataset: Latitude and Longitude points
data = {
    'latitude': asal_latitude,  # Random latitudes
    'longitude': asal_longitude,  # Random longitudes
}
df = pd.DataFrame(data)

fig = px.density_mapbox(
    df,
    lat="latitude",
    lon="longitude",
    radius=10,  # Adjust for smoothing
    center=dict(lat=-7.0, lon=112.0),  # Set center of the map
    zoom=6,  # Set zoom level
    mapbox_style="carto-positron",  # Choose map style,
    # color_continuous_scale=custom_color_scale  # Custom color scale
)
fig.update_traces(opacity=0.9)
fig.update_layout(
    title={
        "text": "Density Heatmap of Coordinates",  # Your custom title
        "y": 0.95,  # Adjust the vertical position of the title (default is 1.0)
        "x": 0.5,   # Center the title (0.5 is center, 0 is left, 1 is right)
        "xanchor": "center",
        "yanchor": "top",
    },
    title_font=dict(size=20, color="black", family="Arial"),  # Customize font size, color, and family
)



# Display the map
fig.show()


In [105]:
import pandas as pd
import numpy as np
import plotly.express as px

# Example dataset: Latitude and Longitude points
data = {
    'latitude': asal_latitude,  # Random latitudes
    'longitude': asal_longitude,  # Random longitudes
}
df = pd.DataFrame(data)


# Bin data into a 2D histogram
x_bins = np.linspace(df['longitude'].min(), df['longitude'].max(), 10)
y_bins = np.linspace(df['latitude'].min(), df['latitude'].max(), 10)
hist, x_edges, y_edges = np.histogram2d(df['longitude'], df['latitude'], bins=[x_bins, y_bins])

# Assign density values to each point
density = []
for lon, lat in zip(df['longitude'], df['latitude']):
    x_bin = min(max(np.digitize(lon, x_edges) - 1, 0), hist.shape[0] - 1)
    y_bin = min(max(np.digitize(lat, y_edges) - 1, 0), hist.shape[1] - 1)
    density.append(hist[x_bin, y_bin])

df['keterangan'] = density

# Define custom bins and colors
# bins = [0, 1, 3, 6, 11, 17, 27, 65, 119, 1500]
# colors = [
#     "rgb(255, 245, 240)", "rgb(254, 224, 210)", "rgb(252, 187, 161)",
#     "rgb(252, 146, 114)", "rgb(251, 106, 74)", "rgb(239, 59, 44)",
#     "rgb(203, 24, 29)", "rgb(153, 0, 13)"
# ]

# Create density heatmap

custom_color_scale = [
    [0, "rgb(255, 230, 230)"],  # Light red (low density)
    [0.5, "rgb(255, 100, 100)"],  # Mid-range red
    [1, "rgb(255, 0, 0)"]        # Full red (high density)
]
fig = px.density_mapbox(
    df,
    lat="latitude",
    lon="longitude",
    z="keterangan",
    radius=10,
    center=dict(lat=-7.0, lon=112.0),
    zoom=6,
    mapbox_style="carto-positron",
    color_continuous_scale="YlOrRd"
)

fig.update_layout(
    title={
        "text": "Density Heatmap of Coordinates",  # Your custom title
        "y": 0.95,  # Adjust the vertical position of the title (default is 1.0)
        "x": 0.5,   # Center the title (0.5 is center, 0 is left, 1 is right)
        "xanchor": "center",
        "yanchor": "top",
    },
    title_font=dict(size=20, color="black", family="Arial"),  # Customize font size, color, and family
)

fig.show()


In [17]:
df_agg["pair"] = df_agg.apply(lambda row: tuple(sorted([f'{row["asal_latitude"]}|{row["asal_longitude"]}', f'{row["tujuan_latitude"]}|{row["tujuan_longitude"]}'])), axis=1)

In [18]:
df_agg

,kode_titik,asal_tempat,precentage_asal_tempat,tujuan_tempat,precentage_tujuan_tempat,asal_kab_kota,asal_kab_kota_original,tujuan_kab_kota,_count_origin_dest_kab_kota,count_origin_dest_kab_kota,asal_latitude,asal_longitude,tujuan_latitude,tujuan_longitude,pair
419,RSI2,Tempat Kerja,40.65,Tempat Kerja,42.10,BANDUNG,KAB. BANDUNG,BREBES,5,10,-7.10000,107.60000,-7.05000,108.90000,"(-7.05|108.9, -7.1|107.6)"
420,RSI2,Tempat Kerja,40.65,Tempat Kerja,42.10,BANDUNG,KAB. BANDUNG,KOTA SURABAYA,3,10,-7.10000,107.60000,-7.26667,112.71667,"(-7.1|107.6, -7.26667|112.71667)"
421,RSI2,Tempat Kerja,40.65,Tempat Kerja,42.10,BANDUNG,KAB. BANDUNG,KOTA TEGAL,1,10,-7.10000,107.60000,-6.86860,109.11290,"(-6.8686|109.1129, -7.1|107.6)"
422,RSI2,Tempat Kerja,40.65,Tempat Kerja,42.10,BANDUNG,KAB. BANDUNG,KOTA YOGYAKARTA,2,10,-7.10000,107.60000,-7.80000,110.37500,"(-7.1|107.6, -7.8|110.375)"
423,RSI2,Tempat Kerja,40.65,Tempat Kerja,42.10,BANDUNG BARAT,KAB. BANDUNG BARAT,PATI,1,10,-6.83333,107.48333,-6.76667,111.10000,"(-6.76667|111.1, -6.83333|107.48333)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1360,RSI2,Tempat Usaha,3.33,Tempat Usaha,7.04,MALANG,KAB. MALANG,BEKASI,2,10,-8.16667,112.66667,-6.24667,107.10833,"(-6.24667|107.10833, -8.16667|112.66667)"
1361,RSI2,Tempat Usaha,3.33,Tempat Usaha,7.04,SERANG,KAB. SERANG,KOTA SURABAYA,1,10,-6.15000,106.00000,-7.26667,112.71667,"(-6.15|106.0, -7.26667|112.71667)"
1362,RSI2,Tempat Usaha,3.33,Tempat Usaha,7.04,SIDOARJO,KAB. SIDOARJO,TANGERANG,1,10,-7.45000,112.70000,-6.20000,106.46667,"(-6.2|106.46667, -7.45|112.7)"
1363,RSI2,Tempat Usaha,3.33,Tempat Usaha,7.04,TEGAL,KAB. TEGAL,CIREBON,49,50,-7.03333,109.16667,-6.80000,108.56667,"(-6.8|108.56667, -7.03333|109.16667)"


In [11]:
import pandas as pd

# Create the DataFrame
data = {
    "source": ["A", "A", "A", "C", "A", "C", "A", "B"],
    "destination": ["B", "C", "B", "A", "B", "B", "C", "A"],
    "weight": [1, 1, 1, 1, 1, 1, 1, 1],
}
df = pd.DataFrame(data)

# Sort source and destination to treat (A -> B) same as (B -> A)
df["pair"] = df.apply(lambda row: tuple(sorted([row["source"], row["destination"]])), axis=1)
df
# Aggregate the data
# result = df.groupby("pair")["weight"].sum().reset_index()

# # Convert the pair back to readable format
# result["pair"] = result["pair"].apply(lambda x: f"{x[0]} - {x[1]}")

# # Rename columns for clarity
# result = result.rename(columns={"pair": "connection", "weight": "total_weight"})

# # Display the result
# print(result)


,source,destination,weight,pair
0,A,B,1,"(A, B)"
1,A,C,1,"(A, C)"
2,A,B,1,"(A, B)"
3,C,A,1,"(A, C)"
4,A,B,1,"(A, B)"
5,C,B,1,"(B, C)"
6,A,C,1,"(A, C)"
7,B,A,1,"(A, B)"


In [ ]:


asal_latitude = []
asal_longitude = []
tujuan_latitude = []
tujuan_longitude = []
weight = []
_asal_kab_kota = []
_tujuan_kab_kota = []
for d in concatenated_origin_dest_lat_long:
    data = str(d).split('|')
    asal_latitude.append(float(data[0]))
    asal_longitude.append(float(data[1]))
    tujuan_latitude.append(float(data[2]))
    tujuan_longitude.append(float(data[3]))
    weight.append(int(data[4]))
    _asal_kab_kota.append(data[5])
    _tujuan_kab_kota.append(data[6])

df_map_line =  pd.DataFrame({
    'asal_latitude': asal_latitude,
    'asal_longitude': asal_longitude,
    'tujuan_latitude': tujuan_latitude,
    'tujuan_longitude': tujuan_longitude,
    'weight': weight,
    '_asal_kab_kota': _asal_kab_kota,
    '_tujuan_kab_kota': _tujuan_kab_kota,
})
len(tujuan_longitude)

552

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

fig_map_line = px.scatter_mapbox(
    data_frame=df_map_line,
    lat=[*df_map_line['asal_latitude'], *df_map_line['tujuan_latitude']],
    lon=[*df_map_line['asal_longitude'], *df_map_line['tujuan_longitude']],
    hover_name=[*df_map_line['_asal_kab_kota'], *df_map_line['_tujuan_kab_kota']],
    mapbox_style="carto-positron",
    zoom=6.5,
    center={"lat": -7.5, "lon": 111.5},
    title=f"Desire Line Map RSI-9"
)
weight_flag = {}
for i in range(len(df_map_line)):
    weight_flag[df_map_line["weight"][i]] = False if df_map_line["weight"][i] in weight_flag else True
    
    fig_map_line.add_trace(go.Scattermapbox(
        mode="lines",
        lon=[df_map_line['asal_longitude'][i], df_map_line['tujuan_longitude'][i]],
        lat=[df_map_line['asal_latitude'][i], df_map_line['tujuan_latitude'][i]],
        line=dict(width=df_map_line['weight'][i] / 70, color='blue'),
        showlegend=weight_flag[df_map_line["weight"][i]],
        name=f': {df_map_line["weight"][i]}' if weight_flag[df_map_line["weight"][i]] else None
    ))
    
fig_map_line.update_layout(height=800)
fig_map_line.show()